In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the SerperDevTool for web search
search_tool = SerperDevTool()

class AIResearchCrew:
    def __init__(self, topic):
        self.topic = topic

    def run(self):
        # Define Agents

        # Agent 1: Research Manager
        # This agent is responsible for coordinating the research and ensuring all relevant
        # information is gathered. It will delegate specific search tasks.
        self.research_manager = Agent(
            role='Research Manager',
            goal=f'Oversee comprehensive research on "{self.topic}" and ensure all critical information is found and synthesized.',
            backstory=(
                "You are an experienced research manager with a keen eye for detail and the ability to "
                "orchestrate complex information gathering processes. Your expertise lies in ensuring "
                "that no stone is left unturned in the pursuit of knowledge."
            ),
            tools=[search_tool],
            verbose=True,
            allow_delegation=True,
        )

        # Agent 2: Senior Researcher
        # This agent performs the actual web searches and gathers raw data.
        self.senior_researcher = Agent(
            role='Senior Researcher',
            goal=f'Conduct in-depth web research to gather factual information, trends, and key developments related to "{self.topic}".',
            backstory=(
                "You are a highly skilled senior researcher, adept at using search engines "
                "to find precise and relevant information quickly. You prioritize accuracy and "
                "breadth in your findings."
            ),
            tools=[search_tool],
            verbose=True,
        )

        # Agent 3: Data Analyst
        # This agent is responsible for analyzing the raw research data, identifying patterns,
        # and extracting key insights.
        self.data_analyst = Agent(
            role='Data Analyst',
            goal=f'Analyze the gathered research data on "{self.topic}" to identify key insights, trends, and potential implications.',
            backstory=(
                "You are a meticulous data analyst with a talent for transforming raw information "
                "into actionable intelligence. You excel at spotting connections and drawing meaningful "
                "conclusions from complex datasets."
            ),
            verbose=True,
        )

        # Agent 4: Report Writer
        # This agent synthesizes all the information and analysis into a coherent and professional report.
        self.report_writer = Agent(
            role='Report Writer',
            goal=f'Compose a well-structured, comprehensive, and engaging research report on "{self.topic}" based on gathered data and analysis.',
            backstory=(
                "You are a professional report writer, skilled in crafting clear, concise, and compelling "
                "narratives. Your reports are known for their logical flow, informative content, and "
                "professional presentation."
            ),
            verbose=True,
        )

        # Define Tasks

        # Task 1: Research Topic
        # Assigned to the Senior Researcher, delegated by the Research Manager.
        self.research_task = Task(
            description=f"Conduct a thorough web search for the latest information, statistics, "
                        f"and developments regarding '{self.topic}'. Focus on identifying key facts, "
                        f"historical context, current status, and future outlook.",
            expected_output=f"A detailed summary of relevant findings, including URLs of credible sources.",
            agent=self.senior_researcher,
            tools=[search_tool]
        )

        # Task 2: Data Analysis
        # Assigned to the Data Analyst, takes context from the research task.
        self.analysis_task = Task(
            description=f"Review the research findings on '{self.topic}' provided by the researcher. "
                        f"Identify significant trends, recurring patterns, and any emerging challenges or opportunities. "
                        f"Extract key insights that would be valuable for a comprehensive report.",
            expected_output=f"A concise list of key insights, trends, and their potential implications, "
                            f"supported by data points from the research.",
            agent=self.data_analyst,
            context=[self.research_task]
        )

        # Task 3: Draft Report
        # Assigned to the Report Writer, takes context from analysis and research tasks.
        self.draft_report_task = Task(
            description=f"Using the gathered research and analytical insights, draft a comprehensive "
                        f"research report on '{self.topic}'. The report should include:\n"
                        f"- An executive summary highlighting the most critical findings.\n"
                        f"- Detailed sections covering historical context, current status, key trends, and future outlook.\n"
                        f"- A conclusion summarizing the main takeaways and offering potential recommendations or next steps.\n"
                        f"Ensure the report is well-structured, clear, and professional.",
            expected_output=f"A complete, well-formatted research report in markdown format.",
            agent=self.report_writer,
            context=[self.research_task, self.analysis_task]
        )

        # Assemble the Crew

        self.crew = Crew(
            agents=[self.research_manager, self.senior_researcher, self.data_analyst, self.report_writer],
            tasks=[self.research_task, self.analysis_task, self.draft_report_task],
            process=Process.sequential,
            verbose=2
        )

        print(f"Starting the AI Research Crew for topic: '{self.topic}'...\n")
        result = self.crew.kickoff()
        print("\n--- Research Complete! ---")
        print(result)
        return result

if __name__ == "__main__":
    research_topic = "Impact of Artificial Intelligence on the Future of Education"

    research_crew = AIResearchCrew(research_topic)

    final_report = research_crew.run()

